<h1><center>Машинное обучение. Кластеризация новостных текстов</center></h1>
<center>Никиша Ирина, Степачёв Павел, Бакаров Амир</center>
<center>Национальный исследовательский университет "Высшая школа экономики"</center>

<center>Данное исследование посвящено сравению различных алгоритмов векторизации и кластеризации в задаче кластеризации набора новостных текстов. </center>

### Введение

Кластерный анализ (англ. cluster analysis) — многомерная статистическая процедура, выполняющая сбор данных, содержащих информацию о выборке объектов, и затем упорядочивающая объекты в сравнительно однородные группы. Задача кластеризации относится к статистической обработке, а также к широкому классу задач обучения без учителя.

Для проведения экспериментов с кластеризацией новостного корпуса потребуются следующие библиотеки:

In [1]:
# -*- coding: utf-8 -*-
from os import path
import glob
import pickle
import gensim
import numpy as np
import pickle
import time
from nltk.tokenize import RegexpTokenizer
from re import sub
from pandas import DataFrame
from pymystem3 import Mystem
import texterra
from pymorphy2 import MorphAnalyzer
import codecs
import csv
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics import *
from sklearn.cluster import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import *
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
from difflib import SequenceMatcher

Using Theano backend.


In [2]:
API_KEY = '9988cfb979b80264baeba1386cc7e455f99f943c'

morph = MorphAnalyzer()
m = Mystem()
t = texterra.API(API_KEY)
alpha_tokenizer = RegexpTokenizer('\w+')

### Анализ данных

Новостной корпус представлен данными в файлах 'events.csv' и 'raw_news.csv'

In [20]:
df_events = DataFrame.from_csv('events.csv')
df_news = DataFrame.from_csv('raw_news.csv')
texts = list(df_news.text.values)

Данные имеют следующий вид:

In [66]:
df_news.head()

,event_id,text
0,1,В ПЕТЕРБУРГЕ ПРОШЕЛ МИТИНГ ПРОТИВ ПЕРЕДАЧИ ИС...
1,1,"Lenta.co, Москва, 14 января 2017 СИТУАЦИЯ С П..."
2,1,"Аргументы и Факты (aif.ru), Москва, 14 января..."
3,1,"Google Новости ТОП, Москва, 14 января 2017 АК..."
4,1,"Газета.Ru, Москва, 13 января 2017 В МОСКОВСКО..."


Анализ данных:

In [67]:
df_news.event_id.value_counts()

17    102
28    100
18    100
7     100
10    100
12    100
27    100
16    100
1     100
25    100
26    100
21    100
23    100
3      84
22     82
9      82
24     62
4      62
2      51
11     49
19     45
6      41
8      27
13     24
20      8
15      7
5       2
14      2
Name: event_id, dtype: int64

Мы также выяснили, есть ли в корпусе дубликаты, и избавились от них:

In [22]:
df_news[df_news.duplicated()]

,event_id,text
513,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
518,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
522,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
526,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
528,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
536,9,"Коммерсантъ. Новости информ. центра, Москва, ..."


In [23]:
df_news = df_news.drop_duplicates()

### Предобработка текста

Лемматизация (и автоматические приведение к нижнему регистру). Мы решили рассмотреть три варианта лемматизации, поскольку не все способны одинаково хорошо справляться с контекстной омонимией (как, например, во фразе "Поля в поле моет пол и заполнила пол поля"):

In [4]:
def normalize_with_pymorphy(tokens):
    return [morph.parse(word)[0].normal_form for word in tokens]

def normalize_with_mystem(tokens):
    return ''.join(m.lemmatize(' '.join(tokens))).split()

def normalize_with_texterra(tokens):
    time.sleep(10)
    text = ' '.join(tokens)
    return [token[3] for token in list(t.lemmatization(text))[0]]

In [9]:
tricky_text = 'Поля в поле моет пол и заполнила пол поля'
gold_standard_normalized = 'поля в поле мыть пол и заполнить половина поле'

for normalizer, normalize in [('Pymorphy2', normalize_with_pymorphy),
                              ('MyStem', normalize_with_mystem),
                              ('Texterra', normalize_with_texterra)]:
    normalized_text = ' '.join(normalize((alpha_tokenizer.tokenize(tricky_text))))
    print('{}: {:0.2f} ({})'.format(normalizer, SequenceMatcher(None, gold_standard_normalized, normalized_text).ratio(), normalized_text))

Pymorphy2: 0.90 (поль в пол мыть половина и заполнить половина поль)
MyStem: 0.87 (поле в поле мыть пол и заполнять пол поля)
Texterra: 0.88 (поле в поле мывать пол и заполнять пол поле)


Мы представим лемматизированные данные в виде трёх наборов данных и проверим, на каком из них рассмотренные алгоритмы покажут наилучшие результаты:

In [6]:
all_type_of_tokens = {}
tokenizers = {'pymorphy': normalize_with_pymorphy, 'texterra': normalize_with_texterra, 'mystem': normalize_with_mystem}

Данные предварительно сериализованы в .pickle-файлы для того, чтобы впоследствии к ним можно было удобно обращаться:

In [ ]:
for name, tokenizer in tokenizers.items():
    all_texts = []
    for text in texts:
        # print(texts.index(text)) Может, не надо их принтовать?
        text = sub(r'http\S+', '', text)
        tokens = alpha_tokenizer.tokenize(text)
        tokens = tokenizer(tokens)
        all_texts.append(tokens)
    # print(all_texts) Может, не надо их принтовать?
    with open('tokens_from_' + name + '.pickle', 'wb') as f:
        pickle.dump(all_texts, f)

Загружаем наши леммы в словарь типа ```data = {'texterra' : [ [lemma1, lemma2 ...], [], [] ... [] ],  'mystem': [ ], etc.}```:

In [ ]:
data = {}
for filename in glob.iglob(path.join(path.dirname(__file__),'lemmas_from_*.pickle'), recursive=True):
    with open(filename, 'rb') as f:
        data_lemmas = pickle.load(f)
        data[path.splitext(path.basename(filename))[0]] = data_lemmas

### Векторизация

Далее для каждого типа лемм создаем W2V модели, матрицу векторов для всех текстов, сохраняем ее в pickle:

In [ ]:
num_of_features = 200

In [ ]:
for name_of_alg, list_of_docs in data.items():
    
    model = gensim.models.Word2Vec(list_of_docs, size=num_of_features, min_count=30, window=30)
    #model.save(name_of_alg.replace('lemmas','w2v_model2')+'.mdl')
    
    vectors_list = []
    
    for text_id in range(len(list_of_docs)):
        vector_for_each_text = []
        
        for word in list_of_docs[text_id]:
            try:
                featureVec = np.zeros(shape=(1, num_of_features), dtype='float32')
                featureVec = np.add(featureVec, model[word])
                vector_for_each_text.append(featureVec)
                
            except KeyError:
                pass

        first_vector = np.array(vector_for_each_text[0])
        for i in range(1, len(vector_for_each_text)):
            first_vector = np.add(first_vector, vector_for_each_text[i])
            
        resultVec = np.divide(first_vector, len(vector_for_each_text))
        vectors_list.append(resultVec)

    vectors_array = np.array(vectors_list[0])
    for i in range(1, len(vectors_list)):
        vectors_array = np.vstack((vectors_array,vectors_list[i]))

    with open(name_of_alg.replace('lemmas', 'vectors2')+ '.pickle', 'wb') as f:
            pickle.dump(np.matrix(vectors_array), f)

То же самое происходит с Doc2Vec:

In [ ]:
for name_of_alg, list_of_docs in data.items():
    
    #тут короче преобразования для того, чтобы doc2vec нормально кушал тексты 
    sentences = [gensim.models.doc2vec.TaggedDocument(words=list_of_docs[doc], tags=[u'text']) for doc 
                 in range(len(list_of_docs))]
    model = gensim.models.doc2vec.Doc2Vec(sentences, size=num_of_features, min_count=70, window=70)
    #model.save(name_of_alg.replace('lemmas', 'd2v_model') + '.mdl')
    model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


    vectors_list = []
    for text_id in range(len(list_of_docs)):
        vector_for_each_text = []
        for word in list_of_docs[text_id]:
            try:
                featureVec = np.zeros(shape=(1, num_of_features), dtype='float32')
                featureVec = np.add(featureVec, model[word])
                vector_for_each_text.append(featureVec)
            except KeyError:
                pass
        first = np.array(vector_for_each_text[0])

        for i in range(1, len(vector_for_each_text)):
            first = np.add(first, vector_for_each_text[i])
        resultVec = np.divide(first, len(vector_for_each_text))
        vectors_list.append(resultVec)

    vectors_array = np.array(vectors_list[0])

    for i in range(1, len(vectors_list)):
        vectors_array = np.vstack((vectors_array, vectors_list[i]))
        
    with open(name_of_alg.replace('lemmas', 'vectors_d2v') + '.pickle', 'wb') as f:
            pickle.dump(np.matrix(vectors_array), f)

### Кластеризация

Мы используем три алгоритым кластеризации:

In [22]:
list_of_alg = [KMeans, AgglomerativeClustering, SpectralClustering]

Для отрисовки Agglomerative clustering:

In [ ]:
def plotClusters(a):
    z = hac.linkage(a, method='ward')
    hac.dendrogram(z)
    plt.tight_layout()
    plt.show()

Для оценки используем "настоящие" метки новостей из ```raw_news.csv```:

In [80]:
labels = list(df_news.event_id)

Здесь для каждого типа лемм применяем PipeLine:

In [ ]:
for filename in glob.iglob('vectors_d2v_from_*.pickle', recursive=True):
    with open(filename, 'rb') as f:
        data_model = pickle.load(f)
        
        for alg in list_of_alg:
            print(alg, filename.replace('vectors_d2v_from_', '').replace('.pickle',''))
            
            pipeline = Pipeline([('tfidf', TfidfTransformer()),
                                 ('svd', TruncatedSVD(n_components=150)),
                                 ('norm', Normalizer()),
                                 ('clust', alg(n_clusters=28))
                                ])
            pipeline.fit(data_model)
            
            explained_variance = pipeline.named_steps['svd'].explained_variance_ratio_.sum()
            print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
                        
            clust_labels = pipeline.named_steps['clust'].labels_

            print("Homogeneity:", homogeneity_score(labels, clust_labels))
            print("Completeness:", completeness_score(labels, clust_labels))
            print("V-measure",  v_measure_score(labels, clust_labels))
            print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))
            print(confusion_matrix(labels, clust_labels))
            print('==============')
        
        if alg == AgglomerativeClustering:
            plotClusters(data_model)

Вот тут вот сверху confusion matrix, если мы будем рисовать ее...

### Обсуждение

Полученные результаты говорят нам о том, что в большинстве случаев полученные с помощью алгоритмов метки кластеров были похожи на метки кластеров, проставленные людьми; при этом пока ничего не понятно о том, действительно ли полученные кластеры будут интерпретируемы. Для этого мы решеили попробовать разбить данные на 5 кластеров:

In [14]:
with open('tokens_from_pymorphy.pickle', 'rb') as f:
    tokens = pickle.load(f)
words_list = [' '.join([word for word in sentence]) for sentence in tokens]

In [15]:
vectorizer = TfidfVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(words_list)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
U = svd.fit_transform(X) 
clusterizer = SpectralClustering(n_clusters=5)
clusterizer_model = clusterizer.fit(U)

In [16]:
labels = clusterizer_model.labels_

In [17]:
events_merged = []

for event in range(5):
    events = set([df_news.event_id.values[i] for i, x in enumerate(labels) if x == event])
    events_merged.append(events)

In [18]:
for events_set in events_merged:
    for event in events_set:
        print(df_events.loc[df_events.index == event]['name'].values[0])
    print('=======')

Дональд Трамп вступил в должность президента США.
Вышел фильм Навального «он Вам не димон»
Умер Дэвид рокфеллер
теракт произошел в центре Лондона
Юлию Самойлову не пустили на евровидении в Киеве
В центре Киева был убит бывший депутат Госдумы РФ от КПРФ Денис Вороненков
Митинг в москве против коррупции
SpaceX впервые в истории запустила и посадила уже летавшую ракету-носитель
Умер Евгений Евтушенко
Премьер Медведев выступает перед депутатами Госдумы с отчетом об итогах работы правительства за 2016 год
Несанкционированные акции в Москве апрель
Чемпионат мира по хоккею
Победа Макрона во Франции
Митинг против Реновации в Москве
Ураган в Москве
Парламентские выборы в Великобритании
Горячая линия Президента Путина
Кубок конфедерации FiFA
Теракт в Барселоне
Единый день голосования
Дональд Трамп вступил в должность президента США.
CNN показала фильм «Владимир Путин — самый влиятельный человек в мире».
Юлию Самойлову не пустили на евровидении в Киеве
В центре Киева был убит бывший депутат Госду

Вот так вот

### Заключение

Таким образом,